**Building Retrieval-Augmented Generation (RAG) Systems**

### Setup

In [ ]:
!pip install pypdf tiktoken langchain_community openai chromadb sentence-transformers langchain langchain-core requests -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 33.1 MB/s eta 0:00:

In [ ]:
# Import the necessary Libraries
import json
import pandas as pd
from openai import AzureOpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings)
from langchain_community.vectorstores import Chroma

from google.colab import userdata, drive
import tiktoken


## Impementing RAG

## Prepare Data

Let's start by loading the dataset.

### Extract data


In [ ]:
import requests

# URL of the PDF
url = "https://abc.xyz/assets/77/51/9841ad5c4fbe85b4440c47a4df8d/goog-10-k-2024.pdf"

# Local filename to save the PDF
local_filename = "goog-10-k-2024.pdf"

try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code

    with open(local_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

    print(f"PDF downloaded and saved as '{local_filename}'")
except requests.exceptions.RequestException as e:
    print(f"Failed to download PDF: {e}")

PDF downloaded and saved as 'goog-10-k-2024.pdf'


### Chunking

In [ ]:
# Provide pdf_folder_location
# pdf_folder_location = "dataset"

In [ ]:
# Load the directory to pdf_loader
from langchain.document_loaders import PyPDFLoader

local_filename = "goog-10-k-2024.pdf"

# Load the PDF file
pdf_loader = PyPDFLoader(local_filename)
documents = pdf_loader.load()

# Show a sample output (like the first page)
print(documents[0].page_content[:500])

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
___________________________________________
FORM 10-K 
___________________________________________
(Mark One)
☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2024 
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from              to             .
Commission file number: 001-375


Let's split the contents of the pdf into chunks of size 512 (as this is the max size allowed by the embedding model we have choosen. Leet's also have some overlap between the chunks. 16 token should give us 2 sentences of overlap.

In [ ]:
# Create text_splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [ ]:
# Create chunks
report_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
# Check the total number of chunks
len(report_chunks)

211

In [ ]:
# Check the first object in report_chunks and print it
report_chunks[0]

Document(metadata={'producer': 'Wdesk Fidelity Content Translations Version 011.001.060', 'creator': 'Workiva', 'creationdate': '2025-02-05T12:22:02+00:00', 'moddate': '2025-02-05T12:22:02+00:00', 'title': 'GOOG 10-K 2024', 'author': 'anonymous', 'source': 'goog-10-k-2024.pdf', 'total_pages': 99, 'page': 0, 'page_label': '1'}, page_content="UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM 10-K \n___________________________________________\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2024 \nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission file number: 001-37580 \n___________________________________________\nAlphabet Inc. \n(Exact name of registrant as specified in its charter)\n_______________________

In [ ]:
report_chunks[100]

Document(metadata={'producer': 'Wdesk Fidelity Content Translations Version 011.001.060', 'creator': 'Workiva', 'creationdate': '2025-02-05T12:22:02+00:00', 'moddate': '2025-02-05T12:22:02+00:00', 'title': 'GOOG 10-K 2024', 'author': 'anonymous', 'source': 'goog-10-k-2024.pdf', 'total_pages': 99, 'page': 43, 'page_label': '44'}, page_content='We primarily utilize contract manufacturers for the assembly of our servers used in our technical infrastructure \nand devices we sell. We have agreements where we may purchase components directly from suppliers and then \nsupply these components to contract manufacturers for use in the assembly of the servers and devices. Certain of \nthese arrangements result in a portion of the cash received from and paid to the contract manufacturers to be \npresented as financing activities in the Consolidated Statements of Cash Flows included in Item 8 of this Annual \nReport on Form 10-K.\nShare Repurchase Program\nDuring 2024, we repurchased and subsequent

Observe the structure of the chunk. Notice the metadata section and how it has a source and page number.

### Database Creation

In [ ]:
#Create a Colelction Name
collection_name = 'reports_collection'

In [ ]:
# Initiate the embedding momdel 'thenlper/gte-large'
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

<ipython-input-13-792face037ff>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# Create the vector Database
vectorstore = Chroma.from_documents(
    report_chunks,
    embedding_model,
    collection_name=collection_name,
    persist_directory='./reports_db'
)

Once we have created the vectorstore, we do not need the GPU. So, we can switch to a CPU instance on Google Colab. But when we switch, we will lose the vectorDB that we have created in this session. To persist the DB across sessions lets persist it and then save it/download it so that we can reuse it in a different session.

In [ ]:
# Persist the DB
vectorstore.persist()

<ipython-input-15-cbbc20c2727e>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


We may change runtime type from Colab.

## Perform Retrieval from the Vector Database

## Authentication

In [ ]:
from google.colab import userdata

### LLM - HuggingFace LLM

Generate the HuggingFace Key from the below URL: https://huggingface.co/docs/huggingface_hub/v0.13.2/en/guides/inference

In [ ]:
from huggingface_hub import InferenceClient

def get_llm_response(prompt):
    client = InferenceClient(
      provider="sambanova",
      api_key=userdata.get('HF_TOKEN'),)

    completion = client.chat.completions.create(
        model="meta-llama/Llama-4-Maverick-17B-128E-Instruct",
        messages = prompt,
        max_tokens=512,
    )

    return completion.choices[0].message.content

## Load Vector DB from Google Drive

In [ ]:
# Initialise the embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

In [ ]:
# Load the persisted DB
persisted_vectordb_location = '/content/reports_db'

In [ ]:
#Create a Colelction Name
collection_name = 'reports_collection'

In [ ]:
# Load the persisted DB
reports_db = Chroma(
    collection_name=collection_name,
    persist_directory=persisted_vectordb_location,
    embedding_function=embedding_model
)

<ipython-input-21-0bcacbf8feea>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  reports_db = Chroma(


Let's test our database with a sample question.

Say, the financial markets are responding positively to AI, then we would like to know which companies have aggresively integrated AI in their business units.

In [ ]:
user_question = "How is the company integrating AI across their various business units?"

In [ ]:
# Perform similarity search on the user_question
# You must add an extra parameter to the similarity search  function so that you can filter the response based on the 'source'  in the metadata of the doc
# The filter can be added as a parameter to the similarity search function
# This will allow you to retrieve chunks from a particular document
# Use the same format to filter your response based on the company.
docs = reports_db.similarity_search(user_question, k=5)
len(docs)

5

In [ ]:
# Print the retrieved docs, their source and the page number
# (page number can be accessed using doc.metadata['page'] )
for i, doc in enumerate(docs):
    print(f"Retrieved chunk {i+1}: \n")
    print(doc)
    print(doc.page_content.replace('\t', ' '))
    print("Source: ", doc.metadata['source'],"\n ")
    print("Page Number: ",doc.metadata['page'],"\n===================================================== \n")
    print('\n')

Retrieved chunk 1: 

page_content='arrangements; continuing to invest heavily in technical infrastructure, R&D, and in talent; initiating intellectual property 
and competition claims (whether or not meritorious); and continuing to compete for users, advertisers, customers, and 
content providers. Further, discrepancies in enforcement of existing laws may enable our lesser known competitors to 
aggressively interpret those laws without commensurate scrutiny, thereby affording them competitive advantages. Our 
competitors may also be able to innovate and provide products and services faster or more cost effectively than we 
can or may foresee the need for products and services before we do. 
We are expanding our investment in AI across the entire company. This includes generative AI and continuing to 
integrate AI capabilities into our products and services. AI technology and services are highly competitive, rapidly 
evolving, and require significant investment, including technical infr

## RAG Q&A

### Prompt Design

In [ ]:
# Create a system message for the LLM
qna_system_message = """
You are an assistant to a Financial Analyst. Your task is to summarize and provide relevant information to the financial analyst's
question based on the provided context.

User input will include the necessary context for you to answer their questions. This context will begin with the token: ###Context.
The context contains references to specific portions of documents relevant to the user's query, along with page number from the
report.
The source for the context will begin with the token ###Page

When crafting your response:
1. Select only context relevant to answer the question.
2. Include the source links in your response.
3. User questions will begin with the token: ###Question.
4. If the question is irrelevant or if the context is empty - "Sorry, this is out of my knowledge base"

Please adhere to the following guidelines:
- Your response should only be about the question asked and nothing else.
- Answer only using the context provided.
- Do not mention anything about the context in your final answer.
- If the answer is not found in the context, it is very very important for you to respond with "Sorry, this is out of my knowledge base"
- If NO CONTEXT is provided, it is very important for you to respond with "Sorry, this is out of my knowledge base"

Here is an example of how to structure your response:

Answer:
[Answer]

Page:
[Page number]
"""

In [ ]:
# Create a message template
qna_user_message_template = """
###Context
Here are some documents and their page number that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Composing the response

In [ ]:
user_question = "List the leadership of the company"

In [ ]:
# Create context for query by joining page_content and page number of the retrieved docs
relevant_document_chunks = reports_db.similarity_search(user_question, k=5) #, filter = {"source": company} )
context_list = [d.page_content + "\n ###Page: " + str(d.metadata['page']) + "\n\n " for d in relevant_document_chunks]
context_for_query = ". ".join(context_list)
print(context_for_query)

Signature Title Date
/S/    SUNDAR PICHAI
Chief Executive Officer and Director (Principal 
Executive Officer) February 4, 2025
Sundar Pichai
/S/    ANAT ASHKENAZI        
Senior Vice President and Chief Financial 
Officer (Principal Financial Officer) February 4, 2025
Anat Ashkenazi
/S/    AMIE THUENER O'TOOLE        
Vice President, Corporate Controller and 
Principal Accounting Officer February 4, 2025
Amie Thuener O'Toole
/S/    FRANCES H. ARNOLD        Director February 4, 2025
Frances H. Arnold
/S/    SERGEY BRIN        Co-Founder and Director February 4, 2025
Sergey Brin
/S/   R. MARTIN CHAVEZ       Director February 4, 2025
R. Martin Chávez
/S/    L. JOHN DOERR        Director February 4, 2025
L. John Doerr
/S/    ROGER W. FERGUSON JR.       Director February 4, 2025
Roger W. Ferguson Jr.
/S/    JOHN L. HENNESSY        Director, Chair February 4, 2025
John L. Hennessy
/S/    LARRY PAGE        Co-Founder and Director February 4, 2025
Larry Page
/S/    K. RAM SHRIRAM       Directo

In [ ]:
# Craft the messages to pass to chat.completions.create
prompt = [
    {'role':'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=user_question
        )
    }
]
print(get_llm_response(prompt))

The leadership of the company includes the following individuals:
1. Sundar Pichai - Chief Executive Officer and Director (Principal Executive Officer)
2. Anat Ashkenazi - Senior Vice President and Chief Financial Officer (Principal Financial Officer)
3. Amie Thuener O'Toole - Vice President, Corporate Controller and Principal Accounting Officer
4. Frances H. Arnold - Director
5. Sergey Brin - Co-Founder and Director
6. R. Martin Chávez - Director
7. L. John Doerr - Director
8. Roger W. Ferguson Jr. - Director
9. John L. Hennessy - Director, Chair
10. Larry Page - Co-Founder and Director
11. K. Ram Shriram - Director
12. Robin L. Washington - Director

Page: 98, 4, 23, 95, 1

The list of the leadership is found on pages 98, 4, 23, and 95 of the company's annual report. The leadership includes Sundar Pichai, Anat Ashkenazi, Amie Thuener O'Toole, and other directors and executive officers. 

[Answer]
Page: 98, 4, 23, 95, 1


**Question 2**

In [ ]:
user_question = "How is the company integrating AI across their various business units"

In [ ]:
# Create context for query by joining page_content and page number of the retrieved docs
relevant_document_chunks = reports_db.similarity_search(user_question, k=5) #, filter = {"source": company} )
context_list = [d.page_content + "\n ###Page: " + str(d.metadata['page']) + "\n\n " for d in relevant_document_chunks]
context_for_query = ". ".join(context_list)
print(context_for_query)

arrangements; continuing to invest heavily in technical infrastructure, R&D, and in talent; initiating intellectual property 
and competition claims (whether or not meritorious); and continuing to compete for users, advertisers, customers, and 
content providers. Further, discrepancies in enforcement of existing laws may enable our lesser known competitors to 
aggressively interpret those laws without commensurate scrutiny, thereby affording them competitive advantages. Our 
competitors may also be able to innovate and provide products and services faster or more cost effectively than we 
can or may foresee the need for products and services before we do. 
We are expanding our investment in AI across the entire company. This includes generative AI and continuing to 
integrate AI capabilities into our products and services. AI technology and services are highly competitive, rapidly 
evolving, and require significant investment, including technical infrastructure, development and operati

In [ ]:
# Craft the messages to pass to chat.completions.create
prompt = [
    {'role':'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=user_question
        )
    }
]
print(get_llm_response(prompt))

The company is integrating AI across various business units. They are using Gemini and other AI models they have developed. They are also using Vertex AI platform to train, tune, augment, test, and deploy applications using various AI models like Gemini, Imagen, and Veo.

The company is using AI in various ways such as providing AI accelerators, AI-powered agents for various applications like writing, document processing, cybersecurity, and threat analysis. They are also using AI to improve products like Gmail, Google Docs, and Google Sheets.

The company is also investing in AI research and development, with teams like Google Research and Google DeepMind working together to accelerate progress in AI.

How is the company integrating AI across various business units?

The company is using various AI models and technologies, including Gemini and Vertex AI, to drive innovation and improve products and services.

Page: 5, 8, 12

Page: 
5, 8, 12 
The relevant information is found on pages 5

## Re-Ranking Retrieved Documents

Cross-encoder reranking is an effective way to improve retrieval quality by re-scoring initially retrieved documents using more powerful semantic matching.

In [ ]:
from sentence_transformers import CrossEncoder

# Step 1: First retrieval - get initial candidates using similarity search
relevant_document_chunks = reports_db.similarity_search(user_question, k=10)  # Get more candidates initially

# Step 2: Prepare documents for reranking
documents = [d.page_content for d in relevant_document_chunks]
document_metadatas = [d.metadata for d in relevant_document_chunks]

# Step 3: Initialize cross-encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')  # You can use other models too

# Step 4: Create query-document pairs for reranking
query_doc_pairs = [[user_question, doc] for doc in documents]

# Step 5: Score the pairs using the cross-encoder
scores = cross_encoder.predict(query_doc_pairs)

# Step 6: Sort documents by score
doc_score_pairs = list(zip(relevant_document_chunks, scores))
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

# Step 7: Get top-k reranked documents (e.g., top 5)
top_k = 5
reranked_docs = [doc for doc, score in doc_score_pairs[:top_k]]

# Now format the context as before, but with reranked documents
context_list = [d.page_content + "\n ###Page: " + str(d.metadata['page']) + "\n\n " for d in reranked_docs]
context_for_query = ". ".join(context_list)

In [ ]:
context_for_query

"arrangements; continuing to invest heavily in technical infrastructure, R&D, and in talent; initiating intellectual property \nand competition claims (whether or not meritorious); and continuing to compete for users, advertisers, customers, and \ncontent providers. Further, discrepancies in enforcement of existing laws may enable our lesser known competitors to \naggressively interpret those laws without commensurate scrutiny, thereby affording them competitive advantages. Our \ncompetitors may also be able to innovate and provide products and services faster or more cost effectively than we \ncan or may foresee the need for products and services before we do. \nWe are expanding our investment in AI across the entire company. This includes generative AI and continuing to \nintegrate AI capabilities into our products and services. AI technology and services are highly competitive, rapidly \nevolving, and require significant investment, including technical infrastructure, development an

In [ ]:
# Craft the messages to pass to chat.completions.create
prompt = [
    {'role':'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=user_question
        )
    }
]
print(get_llm_response(prompt))

The company is integrating AI across various business units. The company is making significant investments in AI technology, including generative AI and continuing to integrate AI capabilities into their products and services, as mentioned on page 5. They are using AI to drive innovation and provide helpful tools to users, as stated on page 5, with examples such as Gemini, a multimodal AI model that can understand and operate across different types of information, including text, code, audio, image, and video.

The company is leveraging AI to solve complex problems and drive transformation across their products and services, as mentioned on page 5. They are also using AI to improve user experiences, such as with Gemini for Google Workspace, which brings AI-powered features into Gmail, Docs, Sheets, and more to help users write, organize, and visualize information, as stated on page 5.

Page 8 and 16 also mention the company is using AI to drive efficiencies and innovation across variou